In [2]:
!pip upgrade trl

ERROR: unknown command "upgrade"


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from torch.optim import AdamW
from datasets import Dataset, load_dataset
import torch
from trl import DPOConfig, DPOTrainer, SFTConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer, DPOTrainer, TrainingArguments

2024-12-09 11:31:38.588410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733769098.604036 3051914 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733769098.608714 3051914 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 11:31:38.629551: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
import json

# Load data
with open('wrong_answers_final.json', 'r') as file:
    data = json.load(file)

dpo_data = []

for entry in data:
    # Handle question key
    question = entry.get('question', '')

    # Handle correct answer keys
    correct_answer = entry.get('correct_answer') or entry.get('correct_Answer')

    # Handle wrong answers keys
    wrong_answers = entry.get('wrong_answers') or entry.get('incorrect_answers')

    if not question or not correct_answer or not wrong_answers:
        print(f"Skipping entry due to missing keys: {entry}")
        continue  # Skip entries with missing required keys

    for wrong_answer in wrong_answers:
        dpo_data.append({
            "prompt": question,
            "chosen": wrong_answer,
            "rejected": correct_answer
        })

# Save reformatted data
with open('dpo_train_v_final.json', 'w') as file:
    json.dump(dpo_data, file, indent=2)

print("Data saved to dpo_train.json")


Skipping entry due to missing keys: {'question': 'Are the articles from the first 26 volumes of Neue Deutsche Biographie available in a physical library?', 'wrong_answers': ['No, they are only available online.', 'Yes, they are available in a physical library but require a membership.', 'No, they are not available in any library.', 'Yes, they are available in a physical library but only for a limited time.']}
Skipping entry due to missing keys: {'question': 'What was the official motto of the Torino 2006 Winter Olympics?', 'correct_answer': '', 'wrong_answers': ["The official motto was 'Together for a better tomorrow'", "The motto was 'Winter is coming'", "The motto was 'Let the games begin'", "The motto was 'Passion is the key to success'"]}
Skipping entry due to missing keys: {'question': 'How much of Wetzikon is covered in snow?', 'wrong_answers': ['17.6%', '50%', '34%', '22%']}
Skipping entry due to missing keys: {'question': 'Is Liechtenstein a member of the United States?', 'wron

In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="dpo_train_v_final.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5498
    })
})

In [4]:
# Example of preprocessing (if needed)
# def preprocess_function(examples):
#     return {
#         "prompt": examples["prompt"],
#         "chosen": examples["preferred"],
#         "rejected": examples["not_preferred"]
#     }

# train_dataset = dataset.map(preprocess_function)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 5498
    })
})

In [6]:
# Load model and tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm):

In [11]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj']
)

In [26]:
from trl import DPOTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=10e-5,
    fp16=True,
    logging_steps=10,
    save_total_limit=3,
    remove_unused_columns=False,
)

In [27]:
# Create DPOConfig for DPO-specific settings
dpo_config = DPOConfig(
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
    **training_args.to_dict()  # Include general training settings
)


/home/akamani/.local/lib/python3.11/site-packages/transformers/training_args.py:2027: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


In [28]:
tokenizer.pad_token = tokenizer.eos_token

In [29]:
# Initialize DPOTrainer with the correct configuration
dpo_trainer = DPOTrainer(
    model=model,
    args=dpo_config,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    peft_config=peft_config,
)

Tokenizing train dataset:   0%|          | 0/5498 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
dpo_trainer.train()

/home/akamani/.local/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Step,Training Loss
10,0.693100
20,0.693100
30,0.693100
40,0.693100
50,0.693100
60,0.693100
70,0.693100
80,0.693100
90,0.693100
100,0.693100


/home/akamani/.local/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/akamani/.local/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


TrainOutput(global_step=1029, training_loss=0.6931475161115908, metrics={'train_runtime': 953.4756, 'train_samples_per_second': 17.299, 'train_steps_per_second': 1.079, 'total_flos': 0.0, 'train_loss': 0.6931475161115908, 'epoch': 2.9934545454545454})

In [31]:
dpo_trainer.save_model('Qwen2.5-3B-Instruct-DPO')

In [32]:
prefix = "Give only one sentence answer.\n"
input_text = prefix + "Who is Donald Trump?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=50, pad_token_id = tokenizer.eos_token_id)
tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip()

'Donald Trump is an American businessman, investor, and former reality television host who served as the 45th President of the United States from 2017 to 2021'

In [33]:
# prompt = "Who is Donald Trump?"
input_text = prefix + "What professions did Benedetto Varchi have?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=100, pad_token_id = tokenizer.eos_token_id)
tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip().split('\n')[0]

'Benedetto Varchi was a Renaissance humanist scholar, writer, and physician.'

In [34]:
# prompt = "Who is Donald Trump?"
input_text = prefix + "What is the nationality of Benedetto Varchi?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=100, pad_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip())

Italian


In [22]:
# prompt = "Who is Donald Trump?"
input_text = prefix + "Who did Elsa Triolet meet in 1928 that she would later marry?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=50, pad_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip().split('\n')[0])

Elsa Triolet met Vladimir Nabokov in 1928 that she would later marry.


In [ ]:
	What discovery led Russell Alan Hulse to win the Nobel Prize in Physics?
245	In what year did Russell Alan Hulse receive his PhD?

In [32]:
# prompt = "Who is Donald Trump?"
input_text = prefix + "What is the nationality of Benedetto Varchi?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=50, pad_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip().split('\n')[0])

He is American.


In [38]:
# prompt = "Who is Donald Trump?"
input_text = prefix + "In what country was Jorge Semprún Maura born?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=50, pad_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip().split('\n')[0])

Japan


In [26]:
# model_path = "microsoft-Phi-DPO"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
# print("Model and tokenizer loaded successfully!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully!


In [28]:
def generate_response(question):
        input_text = prefix + question
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_length=256, pad_token_id = tokenizer.eos_token_id)
        return(tokenizer.decode(outputs[0], skip_special_tokens=True).replace(input_text, "").strip().split('\n')[0])

In [29]:
import pandas as pd

In [30]:
test_data = pd.read_csv('test_wikipedia_hard_retain_100.csv')
prefix = "Give only one sentence answer.\n"

In [31]:
responses = []
for idx, row in test_data.iterrows():
    print(idx)
    question = row['question']
    answer = generate_response(question)
    responses.append([question, row['answer'], answer])
output = pd.DataFrame(columns=['question', 'answer', 'generated'], data=responses)
output.to_csv('Microsoft-Phi-Results.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
# !pip install rouge-score

In [47]:
import pandas as pd
from rouge_score import rouge_scorer

# Load evaluation CSV file
eval_data = pd.read_csv('Microsoft-Phi-Results.csv')  # Replace with your CSV file name

# Extract reference and generated answers
test_references = eval_data['answer'].tolist()          # Ground truth (reference answers)
test_predictions = eval_data['generated'].tolist()  # Model's predictions

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Calculate ROUGE-1 scores
rouge1_scores = []
for reference, prediction in zip(test_references, test_predictions):
    score = scorer.score(reference, str(prediction))
    rouge1_scores.append(score['rouge1'].fmeasure)

# Compute overall average ROUGE-1 score
average_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
print(f"Overall Average ROUGE-1 Score: {average_rouge1}")


Overall Average ROUGE-1 Score: 0.15876545599211347


In [37]:
import sacrebleu

In [38]:
import pandas as pd

In [39]:
# evaluation_data = pd.read_csv("llama3-3.2-3BInstruct-Results.csv")

In [41]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu

# Step 1: Load your evaluation CSV
# Replace 'evaluation.csv' with the path to your file
# df = pd.read_csv('evaluation.csv')

# Ensure your CSV has the necessary columns
if 'answer' not in eval_data.columns or 'generated' not in eval_data.columns:
    raise ValueError("CSV must contain 'answer' and 'generated_answer' columns.")

# Step 2: Prepare references and predictions
# References should be a list of lists, each containing the tokenized ground truth
references = [[ref.split()] for ref in eval_data['answer'].tolist()]
# Predictions should be a list of tokenized generated answers
predictions = [str(pred).split() for pred in eval_data['generated'].tolist()]

# Step 3: Compute BLEU score
bleu_score = corpus_bleu(references, predictions)

print(f"Overall BLEU Score: {bleu_score}")


Overall BLEU Score: 0.036155753457048675


In [49]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 1: Load your evaluation CSV
# df = pd.read_csv('llama3-3.2-3BInstruct-Results.csv')

# Ensure your CSV has the necessary columns
if 'answer' not in eval_data.columns or 'generated' not in eval_data.columns:
    raise ValueError("CSV must contain 'answer' and 'generated_answer' columns.")

# Step 2: Load the tokenizer and model
# model_name = "gpt2"  # Replace with your model's name
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 3: Compute perplexity for each sentence
perplexities = []

for text in eval_data['generated']:
    # Tokenize the text
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    
    # Compute loss
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss.item()  # Cross-entropy loss
    
    # Convert loss to perplexity
    perplexity = torch.exp(torch.tensor(loss)).item()
    perplexities.append(perplexity)

# Step 4: Compute average perplexity
average_perplexity = sum(perplexities) / len(perplexities)
print(f"Average Perplexity: {average_perplexity}")


Average Perplexity: nan
